# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770847261155                   -0.52    9.0         
  2   -2.772147418672       -2.89       -1.32    1.0    142ms
  3   -2.772170246383       -4.64       -2.44    1.0    107ms
  4   -2.772170642916       -6.40       -3.13    1.0    133ms
  5   -2.772170722791       -7.10       -4.26    2.0    140ms
  6   -2.772170722871      -10.10       -4.37    1.0    112ms
  7   -2.772170723007       -9.87       -5.01    1.0    127ms
  8   -2.772170723013      -11.24       -5.29    1.0    116ms
  9   -2.772170723015      -11.64       -6.24    1.0    124ms
 10   -2.772170723015      -12.95       -6.49    2.0    139ms
 11   -2.772170723015      -14.03       -6.84    1.0    127ms
 12   -2.772170723015      -13.97       -7.93    1.0    129ms
 13   -2.772170723015   +  -14.40       -8.24    2.0    145ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ----

1.7735581147246593

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770702033160                   -0.53    9.0         
  2   -2.772057415916       -2.87       -1.29    1.0    105ms
  3   -2.772082921536       -4.59       -2.70    1.0    140ms
  4   -2.772083416887       -6.31       -4.18    2.0    123ms
  5   -2.772083417788       -9.05       -4.86    2.0    167ms
  6   -2.772083417810      -10.66       -5.87    1.0    112ms
  7   -2.772083417811      -11.97       -6.40    2.0    128ms
  8   -2.772083417811      -13.69       -6.89    1.0    144ms
  9   -2.772083417811   +  -13.78       -8.05    1.0    116ms

Polarizability via ForwardDiff:       1.7725349954551168
Polarizability via finite difference: 1.7735581147246593
